# Free Space Path Loss
This library computes free space path loss using the Friis equation. It is instructive to break down the Friis equation into the spherical spreading loss and effective aperture components

In [ ]:
import astropy.units as u

import spacelink.core.path as path

print(f"Spreading Loss (1000 km): {path.spreading_loss(1000 * u.km):.2f}")
print(f"Aperture Loss (10 GHz): {path.aperture_loss(10 * u.GHz):.2f}")
print(
    f"FSPL (1000 km, 10 GHz): {path.free_space_path_loss(1000 * u.km, 10 * u.GHz):.2f}"
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

distances = [100, 1000, 10000, 100000, 384400] * u.km
freqs = [1, 2.25, 8.4, 32] * u.GHz

print("Path Loss Component Values (dB)")
print("-" * 75)
print(f"{'Distance (km)':<15}{'Spreading Loss':<20}")
for d in distances:
    sl = path.spreading_loss(d)
    print(f"{d:<15}{sl:<20.2f}")

print("\n")
print(f"{'Frequency (GHz)':<15}{'Aperture Loss':<20}")
for f in freqs:
    al = path.aperture_loss(f)
    print(f"{f:<15}{al:<20.2f}")

print("\n")
print("Free Space Path Loss (dB) Table")
print("-" * 75)
header = f"{'Distance (km)':<15}"
for f in freqs:
    header += f"{f} GHz{'':<10}"
print(header)
for d in distances:
    row = f"{d:<15}"
    for f in freqs:
        loss = path.free_space_path_loss(d, f)
        row += f"{loss:<15.2f}"
    print(row)

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
dist_range = np.logspace(1, 6, 500)
sp_loss = [path.spreading_loss(d * u.km).value for d in dist_range]
plt.plot(dist_range, sp_loss)
plt.xscale("log")
plt.xlabel("Distance (km)")
plt.ylabel("Spreading Loss (dB)")
plt.title("Spreading Loss vs Distance")
plt.grid(True)

plt.subplot(1, 3, 2)
freq_range = np.logspace(9, 11, 500)
ap_loss = [path.aperture_loss(f * u.Hz).value for f in freq_range]
plt.plot(freq_range / 1e9, ap_loss)
plt.xscale("log")
plt.xlabel("Frequency (GHz)")
plt.ylabel("Aperture Loss (dB)")
plt.title("Aperture Loss vs Frequency")
plt.grid(True)

plt.subplot(1, 3, 3)
dist_range_km = np.logspace(2, 6, 500)
for f in freqs:
    fspl_values = [path.free_space_path_loss(d * u.km, f).value for d in dist_range_km]
    plt.plot(dist_range_km, fspl_values, label=f"{f} GHz")
plt.xscale("log")
plt.xlabel("Distance (km)")
plt.ylabel("Free Space Path Loss (dB)")
plt.title("FSPL vs Distance at Different Frequencies")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()